In [9]:
import yfinance as yf
print(yf.__version__)


0.2.65


In [2]:
import pandas as pd
import numpy as np
import datetime
import pickle
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


In [3]:
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime.today()
df = yf.download("TSLA", start=start, end=end)
df = df.sort_index()
print("Dataset head:\n", df.head())

# Use OHLCV features
features = df[['Open', 'High', 'Low', 'Close', 'Volume']]


df['Return'] = df['Close'].pct_change()
df = df.dropna()

# Features (X) = OHLCV, Target (y) = Return
X = features.iloc[1:].values   # drop first row (due to NaN return)
y = df['Return'].values        # returns

C:\Users\navee\AppData\Local\Temp\ipykernel_17420\2509901305.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("TSLA", start=start, end=end)
[*********************100%***********************]  1 of 1 completed

Dataset head:
 Price           Close       High        Low       Open    Volume
Ticker           TSLA       TSLA       TSLA       TSLA      TSLA
Date                                                            
2015-01-02  14.620667  14.883333  14.217333  14.858000  71466000
2015-01-05  14.006000  14.433333  13.810667  14.303333  80527500
2015-01-06  14.085333  14.280000  13.614000  14.004000  93928500
2015-01-07  14.063333  14.318667  13.985333  14.223333  44526000
2015-01-08  14.041333  14.253333  14.000667  14.187333  51637500


In [4]:
scaler_X = MinMaxScaler(feature_range=(0,1))
scaled_X = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler(feature_range=(0,1))
scaled_y = scaler_y.fit_transform(y.reshape(-1,1))

# Save scalers
with open("scaler_X.pkl","wb") as f:
    pickle.dump(scaler_X,f)
with open("scaler_y.pkl","wb") as f:
    pickle.dump(scaler_y,f)


In [5]:
print(len(scaled_X))

2703


In [6]:

training_data_len = int(len(scaled_X) * 0.8)
train_X, test_X = scaled_X[:training_data_len], scaled_X[training_data_len:]
train_y, test_y = scaled_y[:training_data_len], scaled_y[training_data_len:]


def create_sequences(X, y, look_back=60):
    x_seq, y_seq = [], []
    for i in range(look_back, len(X)):
        x_seq.append(X[i-look_back:i])
        y_seq.append(y[i])
    return np.array(x_seq), np.array(y_seq)

look_back = 60
x_train, y_train = create_sequences(train_X, train_y, look_back)
x_test, y_test = create_sequences(test_X, test_y, look_back)

print("x_train shape:", x_train.shape)


x_train shape: (2102, 60, 5)


In [7]:
h=tf.keras.losses.MeanSquaredError

In [8]:
model = Sequential([
    LSTM(130, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])),
    Dropout(0.2),
    LSTM(130, return_sequences=False),
    Dropout(0.2),
    Dense(100, activation='relu'),
    Dense(1)  
])

model.compile(optimizer="adam", loss="MeanSquaredError")

history = model.fit(
    x_train, y_train,
    batch_size=32,
    epochs=50,
    validation_split=0.1,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)

# Save Model
model.save("model_returns.h5")


c:\Users\navee\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 10s 102ms/step - loss: 0.0260 - val_loss: 0.0079
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 5s 86ms/step - loss: 0.0079 - val_loss: 0.0085
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 95ms/step - loss: 0.0080 - val_loss: 0.0077
Epoch 4/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 98ms/step - loss: 0.0077 - val_loss: 0.0080
Epoch 5/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 94ms/step - loss: 0.0075 - val_loss: 0.0092
Epoch 6/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 104ms/step - loss: 0.0073 - val_loss: 0.0083
Epoch 7/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 97ms/step - loss: 0.0073 - val_loss: 0.0081
Epoch 8/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 6s 100ms/step - loss: 0.0072 - val_loss: 0.0082
